In [1]:
import numpy as np
import pandas as pd
from numpy import linalg as la
pu = (1-1/1.1)/(1.1 - 1/1.1) #risk-netural probability: pu and pd
pd = 1 - pu
pu

0.47619047619047605

In [45]:
price_tree = np.zeros((53,53)) # the price change of 1 unit of gasoline in the next 52 weeks.
for i in range(53):
    for j in range(i+1):
        price_tree[i,j] = float((1.1**(2*j - i))) 
final_nodes = price_tree[52]
one_up = np.zeros((53,53))
one_up[52] =  np.array([max(i - 1, 0) for i in final_nodes])
for i in reversed(range(52)):
    for j in range(i+1):
        one_up[i,j] = pd*one_up[i+1, j] + pu*one_up[i+1, j+1]


coordinates = []
for i in range(27,53):
    coordinates.append([52,i])

two_up = np.zeros((53,53))
two_up[52] = one_up[52]

for i in reversed(range(52)):
    for j in range(i+1):
        two_up[i,j] = max(pd*two_up[i+1, j] + pu*two_up[i+1, j+1], 
                          value_tree[i,j] - 1 + one_up[i,j])
        if i==51 and pd*two_up[i+1, j] + pu*two_up[i+1, j+1] <= value_tree[i,j] - 1 + one_up[i,j]:
            coordinates.append([51,j])
        
        
three_up = np.zeros((53,53))
three_up[51:] = two_up[51:]


for i in reversed(range(51)):
    for j in range(i+1):
        three_up[i,j] = max(pd*three_up[i+1, j] + pu*three_up[i+1, j+1], value_tree[i,j] - 1 + two_up[i,j])
        if i==50 and pd*two_up[i+1, j] + pu*two_up[i+1, j+1] <= value_tree[i,j] - 1 + two_up[i,j]:
            coordinates.append([50,j])

        
    
four_up = np.zeros((53,53))
four_up[50:] = three_up[50:]

        
for i in reversed(range(50)):
    for j in range(i+1):
        four_up[i,j] = max(pd*four_up[i+1, j] + pu*four_up[i+1, j+1], value_tree[i,j] - 1 + three_up[i,j])
        if pd*four_up[i+1, j] + pu*four_up[i+1, j+1] <= value_tree[i,j] - 1 + three_up[i,j]:
            coordinates.append([i,j])
four_up_p = four_up[0,0]

In [46]:
four_up_p*50# price of the protaction plan

53.01463475161445

In [52]:
optimal_call = []
for i in coordinates:
    if i[0] >= 49:
        k = [i[0]]
        k.append(round(price_tree[i[0],i[1]],3))
        optimal_call.append(k)
optimal_call #102

'1.0, 1.21, 1.464, 1.772, 2.144, 2.594, 3.138, 3.797, 4.595, 5.56, 6.727, 8.14, 9.85, 11.918, 14.421, 17.449, 21.114, 25.548, 30.913, 37.404, 45.259, 54.764, 66.264, 80.18, 97.017, 117.391, '

In [58]:
one_down = np.zeros((53,53))
one_down[52] = np.array([max(1 - i, 0) for i in final_nodes])

for i in reversed(range(52)):
    for j in range(i+1):
        one_down[i,j] = pd*one_down[i+1, j] + pu*one_down[i+1, j+1]

node_tree = np.zeros((53,53))
coord = []
for i in range(26):
    coord.append([52,i])
    node_tree[52,i] = 1

two_down = np.zeros((53,53))
two_down[52] = one_down[52]

for i in reversed(range(52)):
    for j in range(i+1):
        two_down[i,j] = max(pd*two_down[i+1, j] 
                            + pu*two_down[i+1, j+1]
                            , 1-value_tree[i,j] + one_down[i,j])
        if i == 51 and pd*two_down[i+1, j] + pu*two_down[i+1, j+1] <= 1- value_tree[i,j] + one_down[i,j]:
            coord.append([i,j])
            node_tree[i,j] = 1
        
        
three_down = np.zeros((53,53))
three_down[51:] = two_down[51:]


for i in reversed(range(51)):
    for j in range(i+1):
        three_down[i,j] = max(pd*three_down[i+1, j] + pu*three_down[i+1, j+1],1 - value_tree[i,j] + two_down[i,j])
        if i == 50 and pd*two_down[i+1, j] + pu*two_down[i+1, j+1] <= 1-  value_tree[i,j]  + two_down[i,j]:
            coord.append([i,j])
            node_tree[i,j] = 1

        
    
four_down = np.zeros((53,53))
four_down[50:] = three_down[50:]

        
for i in reversed(range(50)):
    for j in range(i+1):
        four_down[i,j] = max(pd*four_down[i+1, j] + pu*four_down[i+1, j+1], 1 - value_tree[i,j] + three_down[i,j])
        if pd*four_down[i+1, j] + pu*four_down[i+1, j+1] <=  1 - value_tree[i,j]  + three_down[i,j]:
            coord.append([i,j])
            node_tree[i,j] = 1
four_down_p = four_down[0,0]

In [59]:
four_down_p*50000

53014.6347516147

In [70]:
optimal_put = []
for i in coord:
    if i[0] >= 49:
        k = [i[0]]
        k.append(round(price_tree[i[0],i[1]],3))
        optimal_put.append(k)
optimal_put #103

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])